In [40]:
from datetime import datetime
import pandas as pd
import yfinance

def filter_dates(dates, start, end):
    filtered_dates = []
    for obs in dates:
        d = obs.date()
        if d >= start and d <= end:
            filtered_dates.append(obs)
    return pd.DatetimeIndex(filtered_dates)


def get_historical_data(ticker, start="1900-01-01", end=datetime.today().strftime("%Y-%m-%d"), frequency='1d'):
    start_date = datetime.strptime(start, "%Y-%m-%d").date()
    end_date = datetime.strptime(end, "%Y-%m-%d").date()
    if start_date > end_date:
        raise Exception("start date cannot be before end date")
    
    asset = yfinance.Ticker(ticker)
    stock_history = asset.history(period="max", interval=frequency)
    filtered_data = stock_history.loc[filter_dates(stock_history.index.tz_convert(
        "CET"), start_date, end_date)]["Close"]
    return filtered_data

def load_USDCHF_historical_data(start="1900-01-01", end=datetime.today().strftime("%Y-%m-%d")):
    start_date = datetime.strptime(start, "%Y-%m-%d").date()
    end_date = datetime.strptime(end, "%Y-%m-%d").date()
    if start_date > end_date:
        raise Exception("start date cannot be before end date")
    data = pd.read_excel("USDCHF.xlsx").sort_values(by="Date", ascending=True)
    data["Date"] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
    data.set_index("Date", inplace=True, drop=True)
    return data.loc[filter_dates(data.index, start_date, end_date)]["Close"]

####################################################
#                                                  #
#   Note:                                          #
#                                                  #
#   1. SPX INDEX is mapped to the ticker `^GSPC`   #
#   2. We used USD/CHF directly from Bloomberg as  # 
#      it went back to 1971, instead our API       #
#      only went to 2003. However, the mapping     #
#      to the ticker in the API is `CHFUSD=X`      #
#                                                  #
####################################################

print("SPX DATA:")
print(get_historical_data("^GSPC", "2001-05-01", "2001-05-31"))

print("CHF/USD DATA:")
print(load_USDCHF_historical_data("2001-05-01", "2001-05-31"))


SPX DATA:
2001-05-01 00:00:00-04:00    1266.439941
2001-05-02 00:00:00-04:00    1267.430054
2001-05-03 00:00:00-04:00    1248.579956
2001-05-04 00:00:00-04:00    1266.609985
2001-05-07 00:00:00-04:00    1263.510010
2001-05-08 00:00:00-04:00    1261.199951
2001-05-09 00:00:00-04:00    1255.540039
2001-05-10 00:00:00-04:00    1255.180054
2001-05-11 00:00:00-04:00    1245.670044
2001-05-14 00:00:00-04:00    1248.920044
2001-05-15 00:00:00-04:00    1249.439941
2001-05-16 00:00:00-04:00    1284.989990
2001-05-17 00:00:00-04:00    1288.489990
2001-05-18 00:00:00-04:00    1291.959961
2001-05-21 00:00:00-04:00    1312.829956
2001-05-22 00:00:00-04:00    1309.380005
2001-05-23 00:00:00-04:00    1289.050049
2001-05-24 00:00:00-04:00    1293.170044
2001-05-25 00:00:00-04:00    1277.890015
2001-05-29 00:00:00-04:00    1267.930054
2001-05-30 00:00:00-04:00    1248.079956
2001-05-31 00:00:00-04:00    1255.819946
Name: Close, dtype: float64
CHF/USD DATA:
2001-05-01    1.7263
2001-05-02    1.7263
2001